In [1]:
import os
from tqdm import tqdm
import argparse

import sys
sys.path.append(os.path.dirname(os.path.abspath(os.path.dirname("sample_train.ipynb"))))
from common.parser import yaml_parser
from common.recoder import save_checkpoint
from common.cocoeval4yolo import CocoEval
from common.utils import compress_label_map
from data.yolo_dataset import *
from model.loss import YOLOv3Loss
from model.model import Darknet4YOLOv3

import torch
import torch.nn
from torch.utils.tensorboard import SummaryWriter



In [2]:
def train(
        model,
        train_loader,
        loss_func,
        optimizer,
        optim_option,
        model_option,
        device,
        epoch,
        logger
        ):
    model.train()

    scales = torch.tensor(model_option["YOLOv3"]["SCALES"]).to(device)       ## [13, 26, 52]
    anchors = torch.tensor(model_option["YOLOv3"]["ANCHORS"]).to(device)

    for i, batch_input in enumerate(tqdm(train_loader, desc="train")):
        n_iteration = (optim_option["OPTIMIZER"]["ITERS_PER_EPOCH"] * epoch) + i

        batch_img = batch_input["img"].to(device)
        batch_label = [label.to(device) for label in batch_input["label_map"]]
        
        #################
        ##  FORWARDING ##
        #################
        pred = model(batch_img)                                                       ### batch_img: tensor(   N, 3, 608, 608) . . . . . . . . . . . N = batch_size
        loss = ( loss_func(pred[0], batch_label[0], scales[0], anchors=anchors[0])    ######## pred: tensor(3, N, 3, S, S, 1 + 4 + class_offset) . . S = scale_size
               + loss_func(pred[1], batch_label[1], scales[1], anchors=anchors[1])    # batch_label: tensor(3, N, 3, S, S, 1 + 4 + class_offset)
               + loss_func(pred[2], batch_label[2], scales[2], anchors=anchors[2]) )  ##### anchors: tensor(3,    3,       2) . . . is list of pairs(anch_w, anch_h)
        loss /= 3

        logger.add_scalar('train/loss', loss.item(), n_iteration)

        # print(f"loss: {loss}")

        #################
        ## BACKWARDING ##
        #################
        loss.backward()
        optimizer.step()

        torch.cuda.empty_cache()


In [3]:
def valid(
        model,
        valid_loader,
        model_option,
        device,
        epoch,
        logger
        ):
    model.eval()

    scales = torch.tensor(model_option["YOLOv3"]["SCALES"]).to(device)       ## [19, 38, 76]
    anchors = torch.tensor(model_option["YOLOv3"]["ANCHORS"]).to(device)
    coco_evaluator = CocoEval()

    for i, batch_input in enumerate(tqdm(valid_loader, desc="valid")):
        batch_img = batch_input["img"].to(device)
        batch_input["label_map"] = compress_label_map(batch_input["label_map"], anchors, scales)
        
        pred = model(batch_img)
        coco_evaluator.update(batch_input, pred)

    ## Examine Accuracy
    mean_average_precision = coco_evaluator.eval()
    logger.add_scalar('test/map', mean_average_precision, epoch)

    return mean_average_precision


In [4]:
import easydict

args = easydict.EasyDict({
    "config": "C:/Users/ryyoon/RY_GitHub/YOLO-v3/configs/model/yolov3.cfg",
    # "weight": "C:/Users/ryyoon/RY_GitHub/YOLO-v3/configs/darknet/yolov4.weights",
    "dataset": "C:/Users/ryyoon/RY_GitHub/YOLO-v3/configs/dataset/yolo_dataset.yml",
    "model": "C:/Users/ryyoon/RY_GitHub/YOLO-v3/configs/model/yolo_model.yml",
    "optimizer": "C:/Users/ryyoon/RY_GitHub/YOLO-v3/configs/optimizer/optimizer.yml",
    "weight_save_dir": "C:/Users/ryyoon/RY_GitHub/YOLO-v3/weights"
})


dataset_option = yaml_parser(args.dataset)
model_option = yaml_parser(args.model)
optimizer_option = yaml_parser(args.optimizer)

In [5]:
######################
## BUILD DATALOADER ##
######################
# train_set_num, train_loader, _ = build_DataLoader(dataset_option, model_option, optimizer_option)

train_dataset = YoloDataset(dataset_option, model_option, split="valid")
train_loader = DataLoader(train_dataset, batch_size=optimizer_option["OPTIMIZER"]["BATCH_SIZE"], shuffle=True, collate_fn=collate_fn)
valid_dataset = YoloDataset(dataset_option, model_option, split="valid")
valid_loader = DataLoader(valid_dataset, batch_size=optimizer_option["OPTIMIZER"]["BATCH_SIZE"], shuffle=True, collate_fn=collate_fn)

In [6]:
# device = torch.device('cpu')
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

###########################
## BUILD MODEL & LOSS_fn ##
###########################
# model = DarknetParser(args.config, args.weight)
model = Darknet4YOLOv3(args.config).to(device)
model = torch.nn.DataParallel(model)
loss_function = YOLOv3Loss()

In [7]:
optimizer = torch.optim.Adam(model.parameters(), lr=optimizer_option["OPTIMIZER"]["LR"])
optimizer_option["OPTIMIZER"]["ITERS_PER_EPOCH"] = len(train_dataset) // optimizer_option["OPTIMIZER"]["BATCH_SIZE"]

In [8]:
logger = SummaryWriter()

if not os.path.isdir(args.weight_save_dir):
    os.makedirs(args.weight_save_dir)

In [9]:
epochs = optimizer_option["OPTIMIZER"]["EPOCHS"]
for epoch in range(epochs):
    ###########
    ## TRAIN ##
    ###########
    train(
            model,
            train_loader,
            loss_function,
            optimizer,
            optimizer_option,
            model_option,
            device,
            epoch,
            logger,
            )
    
    ###########
    ## VALID ##
    ###########
    acc = valid(
                    model,
                    valid_loader,
                    model_option,
                    device,
                    epoch,
                    logger
                    )
    
    print(f"Epoch: ({epoch + 1}/{epochs}) . . . [acc: {acc:.2f}]")
    save_checkpoint(epoch,
                    acc,
                    model,
                    optimizer,
                    # scheduler,
                    # scaler,
                    path=args.weight_save_dir
                    )


valid:   0%|          | 0/1000 [00:00<?, ?it/s]C:\Users\ryyoon\anaconda3\envs\torch\lib\site-packages\torch\_tensor.py:579: UserWarning: floor_divide is deprecated, and will be removed in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values.
To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor'). (Triggered internally at  ..\aten\src\ATen\native\BinaryOps.cpp:467.)
  return torch.floor_divide(other, self)
valid: 100%|██████████| 1000/1000 [03:28<00:00,  4.80it/s]


creating index...
index created!
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.66s).
Accumulating evaluation results...
DONE (t=0.08s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.682
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.682
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.682
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.908
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.898
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.908
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.227
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.682
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.682
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.914
 Average Recall     (